In [0]:
%run "/DatabricksMasterclass/Tutorial"

In [0]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

In [0]:
dbutils.widgets.text(name='separator_char', defaultValue='*')

dbutils.widgets.text('var', 'x')

dbutils.widgets.dropdown(name='var', defaultValue='x', choices=['x', 'y', 'z'])

In [0]:
s=dbutils.widgets.get('separator_char')
s

'*'

In [0]:
l=[
    ('Vivek', 'Purbey')
]
sch=('f_name string, l_name string')
df=spark.createDataFrame(l, schema=sch)

df.display()

f_name,l_name
Vivek,Purbey


In [0]:
from pyspark.sql.functions import *

In [0]:
df.withColumn('full_name', concat_ws(s, 'f_name', 'l_name')).display()

f_name,l_name,full_name
Vivek,Purbey,Vivek*Purbey


In [0]:
dbutils.widgets.remove('separator_char')

In [0]:
dbutils.widgets.removeAll()

# DELTA LAKE

In [0]:
df_sales.write.format('parquet')\
        .mode('append')\
        .option('path','abfss://destination@datalakeansh.dfs.core.windows.net/sales')\
        .save()

## Managed VS External Delta Tables

**Database**

In [0]:
%sql
CREATE DATABASE salesDB;

**Managed Table**

In [0]:
%sql
CREATE TABLE salesDB.mantable  
(
  id INT,
  name STRING,
  marks INT
)
USING DELTA  

In [0]:
%sql
INSERT INTO salesDB.mantable 
VALUES
(1,'aa',30),
(2,'bb',33),
(3,'cc',35),
(4,'DD',40)

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql 
select * from salesDB.mantable

In [0]:
%sql
DROP TABLE salesDB.mantable;

**External Table**

In [0]:
%sql
CREATE TABLE salesDB.exttable  
(
  id INT,
  name STRING,
  marks INT 
)
USING DELTA    
LOCATION 'abfss://destination@datalakeansh.dfs.core.windows.net/salesDB/exttable' 

In [0]:
%sql
INSERT INTO salesDB.exttable 
VALUES
(1,'aa',30),
(2,'bb',33),
(3,'cc',35),
(4,'DD',40)

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql 
select * from salesDB.exttable;

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40


## Delta Tables Functionalities

**INSERT**

In [0]:
%sql
INSERT INTO salesDB.exttable 
VALUES
(5,'aa',30),
(6,'bb',33),
(7,'cc',35),
(8,'DD',40)

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from salesdb.exttable

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40
5,aa,30
6,bb,33
7,cc,35
8,DD,40


**DELETE**

In [0]:
%sql
DELETE FROM salesdb.exttable 
WHERE id = 8

num_affected_rows
1


In [0]:
%sql
select * from salesdb.exttable

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40
5,aa,30
6,bb,33
7,cc,35


**DATA VERSIONING**

In [0]:
%sql
DESCRIBE HISTORY salesdb.exttable;  

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2024-11-16T19:34:52Z,2259224524736159,anshlambaaz_gmail.com#ext#@anshlambaazgmail.onmicrosoft.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> true)",null,List(974581933435333),1116-142243-lumpoy84,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 1896, p25FileSize -> 1035, numDeletionVectorsRemoved -> 1, minFileSize -> 1035, numAddedFiles -> 1, maxFileSize -> 1035, p75FileSize -> 1035, p50FileSize -> 1035, numAddedBytes -> 1035)",null,Databricks-Runtime/14.3.x-scala2.12
3,2024-11-16T19:34:46Z,2259224524736159,anshlambaaz_gmail.com#ext#@anshlambaazgmail.onmicrosoft.com,DELETE,"Map(predicate -> [""(id#11650 = 8)""])",null,List(974581933435333),1116-142243-lumpoy84,2,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3428, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 1979, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1422)",null,Databricks-Runtime/14.3.x-scala2.12
2,2024-11-16T19:33:31Z,2259224524736159,anshlambaaz_gmail.com#ext#@anshlambaazgmail.onmicrosoft.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(974581933435333),1116-142243-lumpoy84,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 948)",null,Databricks-Runtime/14.3.x-scala2.12
1,2024-11-16T19:33:23Z,2259224524736159,anshlambaaz_gmail.com#ext#@anshlambaazgmail.onmicrosoft.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(974581933435333),1116-142243-lumpoy84,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 948)",null,Databricks-Runtime/14.3.x-scala2.12
0,2024-11-16T19:33:16Z,2259224524736159,anshlambaaz_gmail.com#ext#@anshlambaazgmail.onmicrosoft.com,CREATE TABLE,"Map(partitionBy -> [], description -> null, isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(974581933435333),1116-142243-lumpoy84,null,WriteSerializable,true,Map(),null,Databricks-Runtime/14.3.x-scala2.12


**TIME TRAVEL**

In [0]:
%sql
RESTORE TABLE salesdb.exttable TO VERSION AS OF 2;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1896,2,1,2,1035,1896


In [0]:
%sql
select * from salesDB.exttable

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40
5,aa,30
6,bb,33
7,cc,35
8,DD,40


**VACUUM**

In [0]:
%sql
VACUUM salesdb.exttable; 

path
abfss://destination@datalakeansh.dfs.core.windows.net/salesDB/exttable


**VACUUM RETAIN 0 HRS**

In [0]:
%sql
VACUUM salesdb.exttable RETAIN 0 HOURS; 

### DELTA Table Optimization

**OPTIMIZE**

In [0]:
%sql
OPTIMIZE salesDB.exttable

path,metrics
abfss://destination@datalakeansh.dfs.core.windows.net/salesDB/exttable,"List(1, 2, List(1041, 1041, 1041.0, 1, 1041), List(948, 948, 948.0, 2, 1896), 0, null, 1, 2, 0, true, 0, 0, 1731787286142, 1731787289498, 4, 1, null, List(0, 0), 3, 3, 191, 0, null)"


In [0]:
%sql
select * from salesdb.exttable

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40
5,aa,30
6,bb,33
7,cc,35
8,DD,40


**ZORDER BY**

In [0]:
%sql
OPTIMIZE salesdb.exttable ZORDER BY (id)

path,metrics
abfss://destination@datalakeansh.dfs.core.windows.net/salesDB/exttable,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1041), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1731787413070, 1731787414555, 4, 0, null, List(0, 0), 3, 3, 0, 0, null)"


In [0]:
%sql
select * from salesdb.exttable

id,name,marks
1,aa,30
2,bb,33
3,cc,35
4,DD,40
5,aa,30
6,bb,33
7,cc,35
8,DD,40


### AUTO LOADER

**Streaming Dataframe**

In [0]:
df = spark.readStream.format('cloudFiles')\
        .option('cloudFiles.format','parquet')\
        .option('cloudFiles.schemaLocation','abfss://aldestination@datalakeansh.dfs.core.windows.net/checkpoint')\
        .load('abfss://alsource@datalakeansh.dfs.core.windows.net')   

In [0]:
df.writeStream.format('delta')\
               .option('checkpointLocation','abfss://aldestination@datalakeansh.dfs.core.windows.net/checkpoint')\
               .option('mergeSchema','true')\
               .trigger(processingTime='5 seconds')\
               .start('abfss://aldestination@datalakeansh.dfs.core.windows.net/data')